# BERT for Multi-Label Classification
Refer post : https://medium.com/@javaid.nabi/building-a-multi-label-text-classifier-using-bert-and-tensorflow-f188e0ecdc5d

In [ ]:
cd /content/drive/My Drive/Pre-processed-Stack-exchange

/content/drive/My Drive/Pre-processed-Stack-exchange


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#!unzip 'uncased_L-12_H-768_A-12 (2)'.zip

Archive:  uncased_L-12_H-768_A-12 (2).zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [ ]:
#!pip uninstall tensorflow==2.2.0

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [ ]:
#!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 30kB/s 
     |████████████████████████████████| 3.8MB 42.2MB/s 
     |████████████████████████████████| 512kB 43.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=a73c275a9c8a82aa80235d0dda896a95ae96a03732f46ceb649beb0472ecc90e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: 

In [ ]:
##install bert if not already done
!pip install bert-tensorflow

In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [ ]:
##use downloaded model, change path accordingly
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
#'/content/drive/My Drive/DecompRC-master/DecompRC/model/uncased_L-12_H-768_A-12'
BERT_VOCAB= '/content/drive/My Drive/Pre-processed-Stack-exchange/uncased_L-12_H-768_A-12/vocab.txt'
#'./uncased_L-12_H-768_A-12/vocab.txt'

#'./uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = '/content/drive/My Drive/Pre-processed-Stack-exchange/uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '/content/drive/My Drive/Pre-processed-Stack-exchange/uncased_L-12_H-768_A-12/bert_config.json'
#'./uncased_L-12_H-768_A-12/bert_config.json'

In [ ]:
# tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
# tokenizer = tokenization.FullTokenizer(
#       vocab_file=BERT_VOCAB, do_lower_case=True)

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [ ]:
##change path accordingly
#train_data_path='./Downloads/train.csv'
#train = pd.read_csv(train_data_path)
train = pd.read_csv('Pre_No_dup_Chunk1.csv', encoding = 'utf-8', lineterminator='\n') #change name here 
#test = pd.read_csv('./Downloads/test.csv')
test = pd.read_csv('ShuffleDataChunk010.csv', encoding = 'utf-8', lineterminator='\n') #change name here 


In [ ]:
train.head()

,Id,Title,Body,Tags
0,1000002,forcin nginx to render uri from rails using pa...,is it possible to force nginx to render some u...,nginx subdomain ruby-on-rails phusion-passenger
1,1000003,forcing 0000 timezone for rfc2822 times in ...,how can i force the time rfc2822 function to s...,ruby time formatting rfc822 rfc2822
2,1000004,forcing windows xp style on a c form based ...,i have written an application using visual stu...,c# visual-studio-2008 themes styles
3,1000005,forcing auto rotate auto portrait landsca...,i have been poking around the javascript api f...,javascript pdf adobe pdf-generation adobe-reader
4,1000006,forcing date to use a locale other than the ...,is there a way to force the nix command date...,bash unix date


In [ ]:
test.head()

,Id,Title,Body,Tags
0,4142442,unable to iterate over an array in hash in per...,please help me resolve this issue pre_package_...,perl
1,773684,do rdlcs require anything different from the b...,so i am new to microsofts reporting system an...,reporting-services ssrs-reports
2,216907,app on facebook mobile gives me 4oh4 404,we have created a facebook app but when openi...,facebook application mobile http-status-code-404
3,2908899,smart device framework highlights,i am thinking about introducing opennetcf is s...,.net compact-framework smart-device-framework
4,1059195,getting subquery returned more than 1 value e...,if i doselect from cte1 c where c grantnu...,sql-server sql-server-2008


In [ ]:
train = train.sample(5000)
test = test.sample(5000)

In [ ]:
train.columns

Index(['Id', 'Title', 'Body', 'Tags'], dtype='object')

In [ ]:
test.columns

Index(['Id', 'Title', 'Body', 'Tags'], dtype='object')

In [ ]:
#Making a dictionary of word tags #TRAIN
train.Tags = train.Tags.apply(lambda x: str(x).split())
train_tag_freq_dict = {}
for tags in train.Tags:
    for tag in tags:
        if tag not in train_tag_freq_dict:
            train_tag_freq_dict[tag] = 0
        else:
            train_tag_freq_dict[tag] += 1

In [ ]:
# Get most common tags
tags_to_use = 50 #was 200
train_tag_freq_dict_sorted = dict(sorted(train_tag_freq_dict.items(), key=lambda x: x[1], reverse=True))
train_final_tags = list(train_tag_freq_dict_sorted.keys())[:tags_to_use]
len(train_final_tags)

50

In [ ]:
# Change tag data to only for final_tags
train_final_tag_data = []
for tags in train.Tags:
    temp = []
    for tag in tags:
        if tag in train_final_tags:
            temp.append(tag)
    train_final_tag_data.append(temp)

In [ ]:
train_final_tag_data

[['objective-c'],
 [],
 [],
 ['c#'],
 ['c#', 'android'],
 [],
 ['linux', 'bash'],
 [],
 [],
 ['java'],
 ['java', 'html'],
 ['c++'],
 ['windows'],
 [],
 [],
 ['asp.net'],
 [],
 [],
 ['android'],
 ['python'],
 [],
 ['python'],
 [],
 ['vb.net'],
 [],
 ['objective-c'],
 [],
 ['linux'],
 ['php', 'javascript'],
 ['web-services'],
 ['java'],
 ['c'],
 [],
 [],
 ['eclipse'],
 ['c#'],
 ['c#', '.net', 'asp.net'],
 ['jquery'],
 ['android'],
 [],
 [],
 ['python'],
 ['c#'],
 ['linux'],
 [],
 ['javascript', 'arrays'],
 ['mysql', 'sql-server'],
 [],
 [],
 ['c++'],
 ['android'],
 [],
 ['c++'],
 ['c#'],
 [],
 ['windows-7'],
 ['sql-server'],
 ['linux', 'bash'],
 ['php', 'css'],
 ['android'],
 ['jquery'],
 ['php', 'html'],
 ['javascript'],
 ['c++'],
 ['android'],
 ['git'],
 ['ruby-on-rails'],
 ['vb.net'],
 ['java'],
 [],
 ['c#', 'wpf', 'winforms'],
 [],
 [],
 ['python'],
 ['java', 'database'],
 ['php'],
 ['android'],
 ['python'],
 ['javascript', 'jquery'],
 ['linux'],
 ['mysql'],
 ['jquery', 'css'],
 ['c#

In [ ]:
#Making a dictionary of word tags #TRAIN
test.Tags = test.Tags.apply(lambda x: str(x).split())
test_tag_freq_dict = {}
for tags in test.Tags:
    for tag in tags:
        if tag not in test_tag_freq_dict:
            test_tag_freq_dict[tag] = 0
        else:
            test_tag_freq_dict[tag] += 1

In [ ]:
# Get most common tags
tags_to_use = 50 #was 200
test_tag_freq_dict_sorted = dict(sorted(test_tag_freq_dict.items(), key=lambda x: x[1], reverse=True))
test_final_tags = list(test_tag_freq_dict_sorted.keys())[:tags_to_use]
len(test_final_tags)

50

In [ ]:
# Change tag data to only for final_tags
test_final_tag_data = []
for tags in test.Tags:
    temp = []
    for tag in tags:
        if tag in test_final_tags:
            temp.append(tag)
    test_final_tag_data.append(temp)

In [ ]:
test_final_tag_data

[[],
 [],
 ['c#'],
 ['php', 'ajax'],
 [],
 ['asp.net-mvc'],
 ['html'],
 [],
 ['iphone', 'objective-c', 'ios'],
 ['ios'],
 ['networking'],
 [],
 ['javascript', 'regex'],
 [],
 [],
 ['html', 'css'],
 ['java'],
 ['c++'],
 ['android'],
 ['asp.net-mvc'],
 [],
 [],
 ['html'],
 ['jquery'],
 ['database'],
 ['javascript', 'jquery'],
 [],
 [],
 ['python'],
 [],
 ['java'],
 ['php', 'mysql', 'database'],
 ['php', 'jquery'],
 ['java'],
 ['ruby'],
 ['python', 'django'],
 ['java'],
 ['c#', 'visual-studio-2010'],
 ['javascript', 'ruby-on-rails', 'ruby'],
 ['jquery'],
 [],
 [],
 [],
 ['c++'],
 ['php'],
 ['c#', 'multithreading'],
 ['sql', 'ruby-on-rails', 'ruby-on-rails-3'],
 [],
 ['java'],
 ['php'],
 ['python'],
 [],
 ['iphone'],
 [],
 ['java'],
 ['xml', 'string'],
 [],
 [],
 ['javascript', 'asp.net'],
 ['osx'],
 ['wcf'],
 [],
 ['android'],
 ['c++'],
 ['jquery', 'html5'],
 ['c'],
 ['java'],
 ['python'],
 ['javascript'],
 ['c#'],
 ['asp.net', 'asp.net-mvc'],
 ['python', 'osx'],
 ['iphone'],
 [],
 ['java

In [ ]:
#One hot encode the ground truth data  TRAIN
from sklearn.preprocessing import MultiLabelBinarizer
train_tag_encoder = MultiLabelBinarizer()
train_tags_encoded = train_tag_encoder.fit_transform(train_final_tag_data)
train_tags_encoded.shape

(5000, 50)

In [ ]:
train_tags_encoded[1].shape

(50,)

In [ ]:
#One hot encode the ground truth data  Test
from sklearn.preprocessing import MultiLabelBinarizer
test_tag_encoder = MultiLabelBinarizer()
test_tags_encoded = test_tag_encoder.fit_transform(test_final_tag_data)
test_tags_encoded.shape

(5000, 50)

In [ ]:
ID = 'Id'
DATA_COLUMN = 'Title'
LABEL_COLUMNS = 'Tags'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
# Define
# train_label_list = train_final_tags #[0, 1]
# test_label_list = test_final_tags

In [ ]:
# ID = 'Id'
# DATA_COLUMN = 'Title'
# LABEL_COLUMNS = ['Tags']
# #['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [ ]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [ ]:
import numpy as np

In [ ]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values): 
        guid = None
        text_a = row[1]
        if labels_available:
            labels = train_tags_encoded[i] #row[2:]
        else:
            labels = [0]*50   #,0,0,0,0,0]
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [ ]:
TRAIN_VAL_RATIO = 0.9
LEN = train.shape[0]
SIZE_TRAIN = int(TRAIN_VAL_RATIO*LEN)

x_train = train[:SIZE_TRAIN]
x_val = train[SIZE_TRAIN:]

train_examples = create_examples(x_train)

In [ ]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            #logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [ ]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [ ]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([50], tf.int64),   # I CHANGED INITALY = [6]
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
train_file = os.path.join('/content/drive/My Drive/BERT_OUTPUT', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

In [ ]:
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****


INFO:tensorflow:***** Running training *****


INFO:tensorflow:  Num examples = 4500


INFO:tensorflow:  Num examples = 4500


INFO:tensorflow:  Batch size = 32


INFO:tensorflow:  Batch size = 32


INFO:tensorflow:  Num steps = 140


INFO:tensorflow:  Num steps = 140


In [ ]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [ ]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        #I ADDED INITIALLY ONE_HOT
        #labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        #one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [ ]:
OUTPUT_DIR = "/content/drive/My Drive/Pre-processed-Stack-exchange/BERT_OUTPUT"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= 50, #len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/Pre-processed-Stack-exchange/BERT_OUTPUT', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0953287cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/Pre-processed-Stack-exchange/BERT_OUTPUT', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0953287cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.


...

Instructions for updating:
Use standard file APIs to delete files with this prefix.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


INFO:tensorflow:Loss for final step: 0.09041238.


INFO:tensorflow:Loss for final step: 0.09041238.


Training took time  1:46:12.561647


In [ ]:
eval_file = os.path.join('/content/drive/My Drive/Pre-processed-Stack-exchange/BERT_OUTPUT', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [ ]:
# This tells the estimator to run through the entire set.
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:**** Trainable Variables ****


...

INFO:tensorflow:Finished evaluation at 2020-06-09-12:32:01


INFO:tensorflow:Saving dict for global step 140: 0 = 0.50000006, 1 = 0.50000024, 10 = 0.5, 11 = 0.50000006, 12 = 0.5000001, 13 = 1.0, 14 = 0.5000001, 15 = 0.50000006, 16 = 0.50000006, 17 = 0.5000002, 18 = 0.5000001, 19 = 0.5000002, 2 = 0.5, 20 = 0.50000006, 21 = 0.421194, 22 = 0.5000002, 23 = 0.3427421, 24 = 0.50000006, 25 = 0.5, 26 = 0.49438277, 27 = 0.5005681, 28 = 0.5, 29 = 0.572449, 3 = 0.50000024, 30 = 0.5000001, 31 = 0.50000006, 32 = 0.50000006, 33 = 0.50000006, 34 = 0.5000001, 35 = 0.5, 36 = 0.5072314, 37 = 0.50000024, 38 = 0.5000002, 39 = 0.5000001, 4 = 0.5964864, 40 = 0.50000006, 41 = 0.50000024, 42 = 0.5000002, 43 = 0.5000001, 44 = 0.50000006, 45 = 0.50000006, 46 = 0.50000024, 47 = 0.5000001, 48 = 0.5000002, 49 = 0.5000001, 5 = 0.5000001, 6 = 0.50000006, 7 = 0.5, 8 = 0.54868776, 9 = 0.5000005, eval_loss = 0.10431888, global_step = 140, loss = 0.10391107


INFO:tensorflow:Saving dict for global step 140: 0 = 0.50000006, 1 = 0.50000024, 10 = 0.5, 11 = 0.50000006, 12 = 0.5000001, 13 = 1.0, 14 = 0.5000001, 15 = 0.50000006, 16 = 0.50000006, 17 = 0.5000002, 18 = 0.5000001, 19 = 0.5000002, 2 = 0.5, 20 = 0.50000006, 21 = 0.421194, 22 = 0.5000002, 23 = 0.3427421, 24 = 0.50000006, 25 = 0.5, 26 = 0.49438277, 27 = 0.5005681, 28 = 0.5, 29 = 0.572449, 3 = 0.50000024, 30 = 0.5000001, 31 = 0.50000006, 32 = 0.50000006, 33 = 0.50000006, 34 = 0.5000001, 35 = 0.5, 36 = 0.5072314, 37 = 0.50000024, 38 = 0.5000002, 39 = 0.5000001, 4 = 0.5964864, 40 = 0.50000006, 41 = 0.50000024, 42 = 0.5000002, 43 = 0.5000001, 44 = 0.50000006, 45 = 0.50000006, 46 = 0.50000024, 47 = 0.5000001, 48 = 0.5000002, 49 = 0.5000001, 5 = 0.5000001, 6 = 0.50000006, 7 = 0.5, 8 = 0.54868776, 9 = 0.5000005, eval_loss = 0.10431888, global_step = 140, loss = 0.10391107


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 140: /content/drive/My Drive/Pre-processed-Stack-exchange/BERT_OUTPUT/model.ckpt-140


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 140: /content/drive/My Drive/Pre-processed-Stack-exchange/BERT_OUTPUT/model.ckpt-140


In [ ]:
output_eval_file = os.path.join('/content/drive/My Drive/Pre-processed-Stack-exchange/BERT_OUTPUT', "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****


INFO:tensorflow:***** Eval results *****


INFO:tensorflow:  0 = 0.50000006


INFO:tensorflow:  0 = 0.50000006


INFO:tensorflow:  1 = 0.50000024


...

INFO:tensorflow:  eval_loss = 0.10431888


INFO:tensorflow:  global_step = 140


INFO:tensorflow:  global_step = 140


INFO:tensorflow:  loss = 0.10391107


INFO:tensorflow:  loss = 0.10391107


In [ ]:
x_test = test[:10] #testing a small sample
x_test = x_test.reset_index(drop=True)
predict_examples = create_examples(x_test,False)

In [ ]:
test_features = convert_examples_to_features(predict_examples, MAX_SEQ_LENGTH, tokenizer)

change default blue color in android holo theme to orange  or anything else 
django admin configuration
how to solve the number of game frame decreased when we use particles   additive texture mapping 
layered architecture and static methods
aligning to the bottom of an li 
adt plugin not showing up in eclipse indigo
abstracting threading and multiprocessing in python 
implementing dijkstra is algorithm using min heap but failed
call a shell script from another shell script via sudo   environment variables 
why can i not disable full range auto focus in newer point and shoot cameras 


In [ ]:
x_test

,Id,Title,Body,Tags
0,489139,change default blue color in android holo them...,i am using holo theme in my app style name a...,"[android, styles, themes]"
1,3674823,django admin configuration,i have got a mental block on this one i have a...,"[django, admin, models]"
2,1692374,how to solve the number of game frame decrease...,the game engine unity3dthe number of game fram...,"[frame, texture, unity3d, alpha, particles]"
3,2081499,layered architecture and static methods,i need suggestion for three layered architectu...,"[.net, layered]"
4,142316,aligning to the bottom of an li,i want to do this without javascript i can not...,"[html, css, html-lists]"
5,54079,adt plugin not showing up in eclipse indigo,i have recently installed fedora 16 and i am t...,"[eclipse, fedora, adt]"
6,82685,abstracting threading and multiprocessing in p...,is there a module that abstract threading and ...,[python]
7,1801021,implementing dijkstra is algorithm using min h...,i am trying to implement dijkstra is algorithm...,"[java, algorithm, graph, shortest-path, dijkstra]"
8,3609831,call a shell script from another shell script ...,i am calling bash script b from script a in s...,"[bash, shell, environment-variables, sudo]"
9,3420966,why can i not disable full range auto focus in...,i owned a number of canon powershot cameras s...,"[autofocus, android, powershot]"


In [ ]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, 50], dtype=tf.int32),
    }) #len(LABEL_COLUMNS)

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [ ]:
print('Beginning Predictions!')
current_time = datetime.now()

predict_input_fn = input_fn_builder(features=test_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)
print(predictions)
print("Prediction took time ", datetime.now() - current_time)

Beginning Predictions!
<generator object Estimator.predict at 0x7f094b70afc0>
Prediction took time  0:00:00.000183


In [ ]:
# for i,value in enumerate(predictions):
#   print(i, value)
#         if value > 0.5:
#             predictions[i] = 1
#         else:
#             predictions[i] = 0
# tags = test_tag_encoder.inverse_transform(np.array([predictions]))

In [ ]:
def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction['probabilities']
        probabilities.append(preds)
        #print(probabilities)
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS
    
    return dff
        

In [ ]:
output_df = create_output(predictions)
merged_df =  pd.concat([x_test, output_df], axis=1)
submission = merged_df.drop(['comment_text'], axis=1)
submission.to_csv("sample_submission0.csv", index=False)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:**** Trainable Variables ****


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


TypeError: ignored

In [ ]:
# def gen_pre(predictions):
#     probabilities = []
#     for (i, prediction) in enumerate(predictions):
#         preds = prediction["probabilities"]
#         probabilities.append(preds)
#     return probabilities
  # if prediction > 0.5:
  #   prediction[i] = 1
  # else:


  # preds = prediction['probabilities']
  #       probabilities.append(preds)
  #       #print(probabilities)

[(sentence, prediction['probabilities']) for sentence, prediction in zip(predict_examples, predictions)]
# [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(predict_examples, predictions)]

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:**** Trainable Variables ****


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


[(<__main__.InputExample at 0x7f094c8c0ac8>,
  array([0.03849465, 0.043286  , 0.05634689, 0.04405656, 0.05000308,
         0.04735276, 0.04797074, 0.08364806, 0.07622784, 0.04169148,
         0.04468185, 0.040815  , 0.04693601, 0.05220434, 0.02790022,
         0.03910238, 0.03657055, 0.02973711, 0.04648602, 0.04144642,
         0.05322158, 0.03510898, 0.03664082, 0.03549328, 0.02820802,
         0.04911071, 0.08529949, 0.07061601, 0.06504124, 0.05092382,
         0.04241371, 0.04792306, 0.05002648, 0.04215783, 0.03733307,
         0.06657854, 0.05548838, 0.03897947, 0.05050904, 0.05327958,
         0.05452967, 0.04366428, 0.03993604, 0.04300839, 0.03615299,
         0.03369543, 0.04068395, 0.04164919, 0.03947237, 0.04613531],
        dtype=float32)),
 (<__main__.InputExample at 0x7f094c8c0588>,
  array([0.0388217 , 0.04318565, 0.05599087, 0.04400915, 0.05007738,
         0.0470888 , 0.04803234, 0.0834398 , 0.07556733, 0.04181683,
         0.04466367, 0.04114768, 0.04675978, 0.05241653,

In [ ]:
def gener(predictions):
  probabilities = []
  for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        for i, value in enumerate(preds):
            if value > 0.5:
               preds[i] = 1
            else:
              preds[i] = 0 
        tags = test_tag_encoder.inverse_transform(np.array([preds]))
        #probabilities.append(preds)
  return tags

In [ ]:
a = gener(predictions)

UnboundLocalError: ignored

In [ ]:
a

[('android',
  'asp.net',
  'bash',
  'c',
  'database',
  'html',
  'jquery',
  'json',
  'linux',
  'multithreading',
  'python',
  'regex',
  'ruby-on-rails',
  'ruby-on-rails-3',
  'sql')]

1

In [ ]:

[(sentence, test_tag_encoder.inverse_transform(prediction['probabilities'])) for sentence, prediction in zip(predict_examples, predictions)]


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:num_labels:50;logits:Tensor("loss/BiasAdd:0", shape=(?, 50), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 50), dtype=float32)


INFO:tensorflow:**** Trainable Variables ****


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


IndexError: ignored

In [ ]:
def getPrediction(in_sentences):
  #labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
predictions = getPrediction()


NameError: ignored

In [ ]:
submission.head()

##### Thanks for Reading, most of the code is from below examples

https://github.com/google-research/bert/blob/master/run_classifier.py

https://github.com/yajian/bert/blob/master/run_multilabels_classifier.py